### Imports

In [2]:
import os
import time
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

### Data Preprocessing

In [3]:
warnings.filterwarnings('ignore')

Loading

In [4]:
path = './drive/MyDrive/Projects/data/DS_2/'
train = pd.read_csv(path + 'bank_data_train.csv')
test = pd.read_csv(path + 'bank_data_test.csv')

In [5]:
target = 'TARGET'

In [6]:
print(train.shape, test.shape)

(355190, 116) (88798, 116)


In [7]:
print(train[target].value_counts(normalize=True))

TARGET
0    0.918565
1    0.081435
Name: proportion, dtype: float64


Splitting

In [8]:
X = train.drop(columns=target)
y = train[target]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

X_test = test.drop(columns=target)
y_test = test[target]

Filling missing values

In [9]:
num_cols = X_train.select_dtypes(exclude='object').columns.tolist()
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_val[num_cols] = num_imputer.transform(X_val[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_val[cat_cols] = cat_imputer.transform(X_val[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])

Encode categorical features with frequency encoding

In [10]:
for col in cat_cols:
    freq = X_train[col].value_counts(normalize=True)
    X_train[col] = X_train[col].map(freq)
    X_val[col] = X_val[col].map(freq).fillna(0)
    X_test[col] = test[col].map(freq).fillna(0)

Handling outliers

In [11]:
for col in num_cols:
    Q1 = X_train[col].quantile(0.25)
    Q3 = X_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    X_train[col] = X_train[col].clip(lower, upper)
    X_val[col] = X_val[col].clip(lower, upper)
    X_test[col] = X_test[col].clip(lower, upper)

Scaling

In [12]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

Feature Selection

In [13]:
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(X_train, y_train)

selector = SelectFromModel(model, prefit=True)

X_train_selected = selector.transform(X_train)
X_val_selected = selector.transform(X_val)
X_test_selected = selector.transform(X_test)

X_train.columns[selector.get_support()]

Index(['ID', 'AMOUNT_RUB_CLO_PRC', 'AMOUNT_RUB_SUP_PRC', 'CLNT_TRUST_RELATION',
       'APP_MARITAL_STATUS', 'REST_AVG_CUR', 'APP_KIND_OF_PROP_HABITATION',
       'CLNT_JOB_POSITION_TYPE', 'AMOUNT_RUB_NAS_PRC', 'CLNT_JOB_POSITION',
       'APP_DRIVING_LICENSE', 'TRANS_COUNT_SUP_PRC', 'APP_EDUCATION',
       'TRANS_COUNT_NAS_PRC', 'APP_TRAVEL_PASS', 'CR_PROD_CNT_TOVR', 'APP_CAR',
       'APP_POSITION_TYPE', 'TRANS_COUNT_ATM_PRC', 'AMOUNT_RUB_ATM_PRC', 'AGE',
       'APP_EMP_TYPE', 'REST_DYNAMIC_CUR_1M', 'APP_COMP_TYPE',
       'REST_DYNAMIC_CUR_3M', 'CNT_TRAN_SUP_TENDENCY3M',
       'TURNOVER_DYNAMIC_CUR_1M', 'SUM_TRAN_SUP_TENDENCY3M',
       'CNT_TRAN_ATM_TENDENCY3M', 'CNT_TRAN_ATM_TENDENCY1M',
       'SUM_TRAN_ATM_TENDENCY3M', 'SUM_TRAN_ATM_TENDENCY1M',
       'TURNOVER_DYNAMIC_CUR_3M', 'PACK', 'CLNT_SETUP_TENOR',
       'TRANS_AMOUNT_TENDENCY3M', 'TRANS_CNT_TENDENCY3M'],
      dtype='object')

### Training Models

Naive Classifier

In [56]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train_selected, y_train)
y_pred_dummy = dummy.predict(X_val_selected)
y_proba_dummy = dummy.predict_proba(X_val_selected)[:, 1]

print("Dummy Classifier metrics:")
print("Accuracy:", accuracy_score(y_val, y_pred_dummy))
print("ROC AUC:", roc_auc_score(y_val, y_proba_dummy))
print(classification_report(y_val, y_pred_dummy))

Dummy Classifier metrics:
Accuracy: 0.918564711844365
ROC AUC: 0.5
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     65253
           1       0.00      0.00      0.00      5785

    accuracy                           0.92     71038
   macro avg       0.46      0.50      0.48     71038
weighted avg       0.84      0.92      0.88     71038



Random Forest

In [68]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [20, 30],
    'max_depth': [5, 10],
    'class_weight': ['balanced', None]
}

grid_rf = GridSearchCV(rf, param_grid, cv=2, scoring='roc_auc', verbose=1)

start_time = time.time()
grid_rf.fit(X_train_selected, y_train)
end_time = time.time()

print(f"Random Forest GridSearch done in {end_time - start_time:.2f}s")
print("Best params:", grid_rf.best_params_)

best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_val_selected)
y_proba_rf = best_rf.predict_proba(X_val_selected)[:, 1]

print("Random Forest metrics:")
print("Accuracy:", accuracy_score(y_val, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_val, y_proba_rf))
print(classification_report(y_val, y_pred_rf))

Fitting 2 folds for each of 8 candidates, totalling 16 fits
Random Forest GridSearch done in 110.65s
Best params: {'class_weight': None, 'max_depth': 10, 'n_estimators': 30}
Random Forest metrics:
Accuracy: 0.9185787888172527
ROC AUC: 0.8050296140727214
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     65253
           1       0.67      0.00      0.00      5785

    accuracy                           0.92     71038
   macro avg       0.79      0.50      0.48     71038
weighted avg       0.90      0.92      0.88     71038



| Metric              | Value  | What it means                                       |
| ------------------- | ------ | --------------------------------------------------- |
| Accuracy            | 0.9186 | \~92% overall correct predictions                   |
| ROC AUC             | 0.8050 | Good discrimination ability (0.5=chance, 1=perfect) |
| Precision (class 0) | 0.92   | Of predicted non-churn, 92% correct                 |
| Recall (class 0)    | 1.00   | Model found almost all non-churn cases              |
| Precision (class 1) | 0.67   | Of predicted churn, 67% correct                     |
| Recall (class 1)    | 0.00   | Model detected almost **no churn cases** (bad)      |


In [65]:
y_test_proba_rf = best_rf.predict_proba(X_test_selected)[:, 1]
# print(f"Test ROC AUC: {roc_auc_score(y_test, y_test_proba_rf):.4f}")

Scikit-learn MLPClassifier

In [13]:
param_grid = {
    'hidden_layer_sizes': [(64,), (128,)],
    'alpha': [1e-4, 1e-3],
    'learning_rate': ['adaptive'],
    'learning_rate_init': [0.001],
    'activation': ['relu'],
    'solver': ['adam']
}

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

grid = GridSearchCV(
    MLPClassifier(max_iter=300, early_stopping=True, random_state=42),
    param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

start_time = time.time()
grid.fit(X_train_selected, y_train)
end_time = time.time()

print(f"MLPClassifier training time: {end_time - start_time:.2f}s")

best_mlp = grid.best_estimator_
y_pred_mlp = best_mlp.predict(X_val_selected)
y_proba_mlp = best_mlp.predict_proba(X_val_selected)[:, 1]

print("MLPClassifier metrics:")
print("Accuracy:", accuracy_score(y_val, y_pred_mlp))
print("ROC AUC:", roc_auc_score(y_val, y_proba_mlp))
print(classification_report(y_val, y_pred_mlp))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
MLPClassifier training time: 104.96s
MLPClassifier metrics:
Accuracy: 0.9185365578985895
ROC AUC: 0.7575291312435776
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     65253
           1       0.38      0.00      0.00      5785

    accuracy                           0.92     71038
   macro avg       0.65      0.50      0.48     71038
weighted avg       0.87      0.92      0.88     71038



Keras (TensorFlow High-Level API)

In [20]:
input_dim = X_train_selected.shape[1]

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

start_time = time.time()
history = model.fit(
    X_train_selected, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val_selected, y_val),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)
end_time = time.time()

print(f"Keras model training time: {end_time - start_time:.2f} seconds")

y_proba_keras = model.predict(X_val_selected).flatten()
y_pred_keras = (y_proba_keras > 0.5).astype(int)

print("Keras MLP metrics:")
print("Accuracy:", accuracy_score(y_val, y_pred_keras))
print("ROC AUC:", roc_auc_score(y_val, y_proba_keras))
print(classification_report(y_val, y_pred_keras))

Epoch 1/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.9139 - loss: 0.2884 - val_accuracy: 0.9186 - val_loss: 0.2537 - learning_rate: 0.0010
Epoch 2/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9188 - loss: 0.2564 - val_accuracy: 0.9186 - val_loss: 0.2503 - learning_rate: 0.0010
Epoch 3/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9184 - loss: 0.2517 - val_accuracy: 0.9186 - val_loss: 0.2459 - learning_rate: 0.0010
Epoch 4/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9183 - loss: 0.2496 - val_accuracy: 0.9186 - val_loss: 0.2452 - learning_rate: 0.0010
Epoch 5/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9189 - loss: 0.2467 - val_accuracy: 0.9186 - val_loss: 0.2435 - learning_rate: 0.0010
Epoch 6/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.9193 - loss: 0.2435 - val_accuracy: 0.9186 - val_loss: 0.2413 - learning_rate: 0.0010
Epoch 7/50
4440/4440 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9

TensorFlow (Low-Level API)

In [23]:
from sklearn.utils import class_weight

In [ ]:
def safe_convert_X(X):
    if hasattr(X, "to_numpy"):
        return X.to_numpy().astype(np.float32)
    else:
        return X.astype(np.float32)

def safe_convert_y(y):
    if hasattr(y, "to_numpy"):
        return y.to_numpy().astype(np.float32).reshape(-1, 1)
    else:
        return y.astype(np.float32).reshape(-1, 1)

X_train = safe_convert_X(X_train)
y_train = safe_convert_y(y_train)
X_val = safe_convert_X(X_val)
y_val = safe_convert_y(y_val)

print("Train class distribution:", np.bincount(y_train.flatten().astype(int)))
print("Val class distribution:", np.bincount(y_val.flatten().astype(int)))

In [ ]:
batch_size = 64

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .shuffle(buffer_size=10000) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
class SimpleMLP(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense1 = tf.Variable(tf.random.normal([X_train.shape[1], 64]), name="dense1_weights")
        self.bias1 = tf.Variable(tf.zeros([64]), name="dense1_bias")

        self.dense2 = tf.Variable(tf.random.normal([64, 32]), name="dense2_weights")
        self.bias2 = tf.Variable(tf.zeros([32]), name="dense2_bias")

        self.out = tf.Variable(tf.random.normal([32, 1]), name="out_weights")
        self.out_bias = tf.Variable(tf.zeros([1]), name="out_bias")

    def __call__(self, x, training=False):
        x = tf.matmul(x, self.dense1) + self.bias1
        x = tf.nn.relu(x)
        if training:
            x = tf.nn.dropout(x, rate=0.3)

        x = tf.matmul(x, self.dense2) + self.bias2
        x = tf.nn.relu(x)
        if training:
            x = tf.nn.dropout(x, rate=0.3)

        x = tf.matmul(x, self.out) + self.out_bias
        return tf.sigmoid(x)

In [22]:
model = SimpleMLP()

y_train_1d = y_train.flatten().astype(int)
weights = class_weight.compute_class_weight(class_weight='balanced',
                                            classes=np.unique(y_train_1d),
                                            y=y_train_1d)
class_weights_tf = {
    int(cls): tf.constant(w, dtype=tf.float32)
    for cls, w in zip(np.unique(y_train_1d), weights)
}
print("Computed class weights:", class_weights_tf)

def loss_fn(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    weights = tf.where(tf.equal(y_true, 1), class_weights_tf[1], class_weights_tf[0])
    loss = -(weights * (y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred)))
    return tf.reduce_mean(loss)

optimizer = tf.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model(x_batch, training=True)
        loss = loss_fn(y_batch, y_pred)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

def validate():
    all_preds = []
    all_labels = []
    val_losses = []
    for x_batch, y_batch in val_ds:
        y_pred = model(x_batch, training=False)
        val_losses.append(loss_fn(y_batch, y_pred).numpy())
        all_preds.append(y_pred.numpy())
        all_labels.append(y_batch.numpy())
    val_loss = np.mean(val_losses)
    all_preds = np.vstack(all_preds).flatten()
    all_labels = np.vstack(all_labels).flatten()
    return val_loss, all_labels, all_preds

# --- Training loop with early stopping ---
epochs = 50
patience = 5
best_val_loss = np.inf
wait = 0

start_time = time.time()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_losses = []
    for x_batch, y_batch in train_ds:
        loss = train_step(x_batch, y_batch)
        train_losses.append(loss.numpy())

    train_loss_avg = np.mean(train_losses)
    val_loss, y_val_true, y_val_pred = validate()

    val_auc = roc_auc_score(y_val_true, y_val_pred)
    val_acc = accuracy_score(y_val_true, y_val_pred > 0.5)
    print(f"Train loss: {train_loss_avg:.4f}, Val loss: {val_loss:.4f}, Val ROC AUC: {val_auc:.4f}, Val Acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        wait = 0
        best_weights = [v.numpy() for v in model.trainable_variables]
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Restore best weights after training ends
for var, best_val in zip(model.trainable_variables, best_weights):
    var.assign(best_val)

# Final evaluation
val_loss, y_val_true, y_val_pred = validate()
y_val_pred_class = (y_val_pred > 0.5).astype(int)

print("Final evaluation on validation set:")
print("Accuracy:", accuracy_score(y_val_true, y_val_pred_class))
print("ROC AUC:", roc_auc_score(y_val_true, y_val_pred))
print(classification_report(y_val_true, y_val_pred_class))

Epoch 1/50
Train loss: 6.4537, Val loss: 5.3809, Val ROC AUC: 0.6543, Val Acc: 0.5650
Epoch 2/50
Train loss: 5.6510, Val loss: 4.7196, Val ROC AUC: 0.6853, Val Acc: 0.6234
Epoch 3/50
Train loss: 3.4375, Val loss: 0.6772, Val ROC AUC: 0.6644, Val Acc: 0.7572
Epoch 4/50
Train loss: 0.7182, Val loss: 0.6354, Val ROC AUC: 0.6955, Val Acc: 0.6827
Epoch 5/50
Train loss: 0.6490, Val loss: 0.6225, Val ROC AUC: 0.7151, Val Acc: 0.5223
Epoch 6/50
Train loss: 0.6289, Val loss: 0.6082, Val ROC AUC: 0.7258, Val Acc: 0.6046
Epoch 7/50
Train loss: 0.6194, Val loss: 0.6175, Val ROC AUC: 0.7337, Val Acc: 0.7014
Epoch 8/50
Train loss: 0.6144, Val loss: 0.6008, Val ROC AUC: 0.7412, Val Acc: 0.6430
Epoch 9/50
Train loss: 0.6114, Val loss: 0.5992, Val ROC AUC: 0.7445, Val Acc: 0.6257
Epoch 10/50
Train loss: 0.6071, Val loss: 0.6077, Val ROC AUC: 0.7460, Val Acc: 0.6927
Epoch 11/50
Train loss: 0.6037, Val loss: 0.6050, Val ROC AUC: 0.7478, Val Acc: 0.6475
Epoch 12/50
Train loss: 0.6005, Val loss: 0.5891, Va

NumPy

In [ ]:
class NumpyMLP:
    def __init__(self, input_dim, hidden1=64, hidden2=32, lr=0.001):
        np.random.seed(42)
        self.lr = lr
        self.W1 = np.random.randn(input_dim, hidden1) * 0.01
        self.b1 = np.zeros((1, hidden1))
        self.W2 = np.random.randn(hidden1, hidden2) * 0.01
        self.b2 = np.zeros((1, hidden2))
        self.W3 = np.random.randn(hidden2, 1) * 0.01
        self.b3 = np.zeros((1, 1))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_deriv(self, x):
        return (x > 0).astype(float)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, X):
        self.Z1 = X @ self.W1 + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = self.A1 @ self.W2 + self.b2
        self.A2 = self.relu(self.Z2)
        self.Z3 = self.A2 @ self.W3 + self.b3
        self.A3 = self.sigmoid(self.Z3)
        return self.A3

    def backward(self, X, y, output):
        m = y.shape[0]
        dZ3 = output - y.reshape(-1, 1)
        dW3 = (self.A2.T @ dZ3) / m
        db3 = np.sum(dZ3, axis=0, keepdims=True) / m

        dA2 = dZ3 @ self.W3.T
        dZ2 = dA2 * self.relu_deriv(self.Z2)
        dW2 = (self.A1.T @ dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * self.relu_deriv(self.Z1)
        dW1 = (X.T @ dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        # Update weights
        self.W3 -= self.lr * dW3
        self.b3 -= self.lr * db3
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs=100, batch_size=64):
        n = X.shape[0]
        for epoch in range(epochs):
            perm = np.random.permutation(n)
            X_shuffled = X[perm]
            y_shuffled = y[perm]
            for i in range(0, n, batch_size):
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]
                output = self.forward(X_batch)
                self.backward(X_batch, y_batch, output)
            if (epoch + 1) % 10 == 0:
                pred = self.forward(X)
                loss = -np.mean(y * np.log(pred + 1e-8) + (1 - y) * np.log(1 - pred + 1e-8))
                print(f"Epoch {epoch + 1}/{epochs} - loss: {loss:.4f}")

    def predict(self, X):
        prob = self.forward(X)
        return (prob > 0.5).astype(int), prob

# Train numpy MLP
np_mlp = NumpyMLP(input_dim=X_train.shape[1], hidden1=64, hidden2=32, lr=0.001)

X_train_np = X_train.values
y_train_np = y_train.values

np_mlp.train(X_train_np, y_train_np, epochs=50, batch_size=128)

X_val_np = X_val.values
y_pred_np, y_proba_np = np_mlp.predict(X_val_np)

print("NumPy MLP metrics:")
print("Accuracy:", accuracy_score(y_val, y_pred_np))
print("ROC AUC:", roc_auc_score(y_val, y_proba_np))
print(classification_report(y_val, y_pred_np))

### Save Final Predictions on Test Set (Using best model)

In [ ]:
# Example: Using Keras model for final prediction (you can choose your best model)

test_proba = model.predict(test_scaled).flatten()
submission = pd.DataFrame({
    'ID': test.index,
    'TARGET': test_proba
})

submission.to_csv('final_predictions.csv', index=False)
print("Saved final predictions to final_predictions.csv")


### Summary table format

In [ ]:
results = pd.DataFrame([
    ['DummyClassifier', 'Most Frequent', '-', accuracy_score(y_val, y_pred_dummy), roc_auc_score(y_val, y_proba_dummy)],
    ['RandomForest', str(grid_search.best_params_), '-', accuracy_score(y_val, y_pred_rf), roc_auc_score(y_val, y_proba_rf)],
    ['MLPClassifier', 'hidden_layer_sizes=(64,32)', '-', accuracy_score(y_val, y_pred_mlp), roc_auc_score(y_val, y_proba_mlp)],
    ['Keras', '64,32 + dropout', 'Adam lr=0.001', accuracy_score(y_val, y_pred_keras), roc_auc_score(y_val, y_proba_keras)],
    ['TensorFlow', '64,32 + dropout', 'Adam lr=0.001', accuracy_score(y_val, y_pred_tf), roc_auc_score(y_val, y_proba_tf)],
    ['NumPy', '64,32 (manual)', 'lr=0.001', accuracy_score(y_val, y_pred_np), roc_auc_score(y_val, y_proba_np)],
], columns=['Library', 'Hyperparameters', 'Notes', 'Accuracy', 'ROC_AUC'])

print(results)
